This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
from pydrake.all import ModelVisualizer, PackageMap, Simulator, StartMeshcat

from manipulation import ConfigureParser, FindResource, running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://ed08c8c5-2e19-44a6-a282-526f554004c1.deepnoteproject.com/7003/


In [ ]:
def AddSpotRemote(parser):
    parser.package_map().AddRemote(
        package_name="spot_description",
        params=PackageMap.RemoteParams(
            urls=[
                f"https://github.com/wrangel-bdai/spot_ros2/archive/20965ef7bba98598ee10878c7b54e6ef28a300c6.tar.gz"
            ],
            sha256=("20a4f12896b04cc73e186cf876bf2c7e905ee88f8add8ea51bf52dfc888674b4"),
            strip_prefix="spot_ros2-20965ef7bba98598ee10878c7b54e6ef28a300c6/spot_description/",
        ),
    )

# Simplified Spot model for mobile manipulation

First we'll use the ModelVisualizer to inspect the model.

In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
AddSpotRemote(visualizer.parser())
visualizer.AddModels(
    url="package://manipulation/spot/spot_with_arm_and_floating_base_actuators.urdf"
)
visualizer.Run(loop_once=not running_as_notebook)
meshcat.DeleteAddedControls()

INFO:drake:PackageMap: Downloading https://github.com/wrangel-bdai/spot_ros2/archive/20965ef7bba98598ee10878c7b54e6ef28a300c6.tar.gz
INFO:drake:Click 'Stop Running' or press Esc to quit


Now we can use HardwareStation to create a basic simulation.

In [ ]:
scenario = load_scenario(
    filename=FindResource(
        "models/spot/spot_with_arm_and_floating_base_actuators.scenario.yaml"
    )
)
station = MakeHardwareStation(scenario, meshcat, parser_preload_callback=AddSpotRemote)
simulator = Simulator(station)
context = simulator.get_mutable_context()
x0 = station.GetOutputPort("spot.state_estimated").Eval(context)
station.GetInputPort("spot.desired_state").FixValue(context, x0)
simulator.AdvanceTo(0.1);

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ed08c8c5-2e19-44a6-a282-526f554004c1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>